In [116]:
import xarray as xr
import os
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import functions.data_wrangling as dw

# PAMIP Models

## Import data

In [ ]:
main_path = '/home/links/ct715/data_storage/PAMIP/processed_monthly/mslp'
models = sorted(os.listdir(main_path))

mslp = {}
for model in models:
    
    # open datasets and run data checker
    file_path = os.path.join(main_path, model, '*.nc')
    ds = xr.open_mfdataset(file_path, concat_dim='ens_ax', combine='nested')
    
    # dodgy ensemble members
    if model == 'HadGEM3-GC31-LL':
        members = np.delete(ds.ens_ax.values, [27,28])
        ds=ds.sel(ens_ax=members)
    
    # take ensemble mean
    ds = ds.resample(time='1ME').mean()
        
    ds['psl'] = ds['psl'] / 100  # convert Pa to hPa
    
    mslp[model] = ds

In [118]:
main_path = '/home/links/ct715/data_storage/PAMIP/processed_daily/daily_efp_mon-avg'
models = sorted(os.listdir(main_path))

daily_efp = {}
for model in models:
    
    # open datasets and run data checker
    file_path = os.path.join(main_path, model, '*.nc')
    ds = xr.open_mfdataset(file_path, concat_dim='ens_ax', combine='nested')
    
    if model == 'HadGEM3-GC31-LL':
        members = np.delete(ds.ens_ax.values, [27,28])
        ds=ds.sel(ens_ax=members)
    
    daily_efp[model] = ds

## Deal with each model individually

In [119]:
save_dir = '/home/links/ct715/data_storage/PAMIP/processed_monthly/daily-efp_mon_u_divFy_psl'

---

### AWI-CM-1-1-MR

- Nice! Lat and Lon match.


In [120]:
# mslp['AWI-CM-1-1-MR']['time'] = daily_efp['AWI-CM-1-1-MR']['time']
# mslp['AWI-CM-1-1-MR'] = mslp['AWI-CM-1-1-MR'].transpose('ens_ax', 'time', 'lat', 'lon', 'bnds')

# awi = xr.merge([daily_efp['AWI-CM-1-1-MR'], mslp['AWI-CM-1-1-MR']])
# awi = awi[['ubar', 'divFy', 'psl']]

# awi.to_netcdf(f'{save_dir}/AWI-CM-1-1-MR_1.1_psl_ubar_divFy_daily-efp_r100.nc')

---

### CanESM5

- Ensembles don't match.... mslp = 300 ens_ax. 
- I've checked and can't seem to find the other 200 ensemble members... so will cut mslp down to the first 100 members. Hopefully that doesn't make too much difference lol

In [121]:
# mslp['CanESM5'] = mslp['CanESM5'].isel(ens_ax=slice(0, 100))  # select first 100 ensemble members
# mslp['CanESM5']['time'] = daily_efp['CanESM5']['time']
# mslp['CanESM5'] = mslp['CanESM5'].transpose('ens_ax', 'time', 'lat', 'lon', 'bnds')

# can = xr.merge([daily_efp['CanESM5'], mslp['CanESM5']])
# can = can[['ubar', 'divFy', 'psl']]

# can.to_netcdf(f'{save_dir}/CanESM5_1.1_psl_ubar_divFy_daily-efp_r100.nc')

---

### CNRM-CM6-1

In [122]:
# mslp['CNRM-CM6-1']['time'] = daily_efp['CNRM-CM6-1']['time']
# mslp['CNRM-CM6-1'] = mslp['CNRM-CM6-1'].transpose('ens_ax', 'time', 'lat', 'lon')

# cnrm = xr.merge([daily_efp['CNRM-CM6-1'], mslp['CNRM-CM6-1']])
# cnrm = cnrm[['ubar', 'divFy', 'psl']]  

# cnrm.to_netcdf(f'{save_dir}/CNRM-CM6-1_1.1_psl_ubar_divFy_daily-efp_r300.nc')

---

### FGOALS-f3-L

In [123]:
# mslp['FGOALS-f3-L']['time'] = daily_efp['FGOALS-f3-L']['time']
# mslp['FGOALS-f3-L'] = mslp['FGOALS-f3-L'].transpose('ens_ax', 'time', 'lat', 'lon', 'bnds')

# fgoals = xr.merge([daily_efp['FGOALS-f3-L'], mslp['FGOALS-f3-L']])
# fgoals = fgoals[['ubar', 'divFy', 'psl']]  

# fgoals.to_netcdf(f'{save_dir}/FGOALS-f3-L_1.1_psl_ubar_divFy_daily-efp_r100.nc')

---

### HadGEM3-GC31-LL

#### Dodgy ensemble members!! Watch out

- Ensemble members don't match: 100 vs 199
- Yep, forgot to include QBOE runs
- Also dail_efp only has 11 months. May 2001 missing, not important for winter months though. Nor SH winter tbf either

In [ ]:
# import functions.eddy_feedback as ef

# efp_nh = ef.calculate_efp(hadll, data_type='pamip')
# efp_sh = ef.calculate_efp(hadll, data_type='pamip', calc_south_hemis=True)

# print(efp_nh)   
# print(efp_sh)

0.1735
0.1729


In [ ]:
# mslp['HadGEM3-GC31-LL'] = mslp['HadGEM3-GC31-LL'].interp(lat=daily_efp['HadGEM3-GC31-LL'].lat, lon=daily_efp['HadGEM3-GC31-LL'].lon, method='linear')
# mslp['HadGEM3-GC31-LL'] = mslp['HadGEM3-GC31-LL'].sel(time=slice('2000-06', '2001-04'))  # select first 100 ensemble members
# mslp['HadGEM3-GC31-LL']['time'] = daily_efp['HadGEM3-GC31-LL']['time']
# mslp['HadGEM3-GC31-LL'] = mslp['HadGEM3-GC31-LL'].transpose('ens_ax', 'time', 'lat', 'lon')

# hadLL = xr.merge([daily_efp['HadGEM3-GC31-LL'], mslp['HadGEM3-GC31-LL']])
# hadLL = hadLL[['ubar', 'divFy', 'psl']]

# hadLL.to_netcdf(f'{save_dir}/HadGEM3-GC31-LL_1.1_psl_ubar_divFy_daily-efp_r197.nc')

#### Wrangling QBOE

- Needed to check variables from previous script.

In [127]:
# path = '/home/links/ct715/data_storage/PAMIP/monthly/1.1_pdSST-pdSIC/psl/HadGEM3-GC31-LL'
# ds = xr.open_mfdataset(f'{path}/psl_Amon_HadGEM3-GC31-LL_pdSST-pdSIC-QBOE-nc*.nc', concat_dim='ens_ax', combine='nested')
# ds

---

### HadGEM3-GC31-MM

- Latitude length doesn't match
- Longitude points don't match
- Time: one is MS and one is ME

In [128]:
# mslp['HadGEM3-GC31-MM'] = mslp['HadGEM3-GC31-MM'].interp(lat=daily_efp['HadGEM3-GC31-MM'].lat, lon=daily_efp['HadGEM3-GC31-MM'].lon, method='linear')
# mslp['HadGEM3-GC31-MM']['time'] = daily_efp['HadGEM3-GC31-MM']['time']
# mslp['HadGEM3-GC31-MM'] = mslp['HadGEM3-GC31-MM'].transpose('ens_ax', 'time', 'lat', 'lon', 'bnds')

# hadmm = xr.merge([daily_efp['HadGEM3-GC31-MM'], mslp['HadGEM3-GC31-MM']])
# hadmm = hadmm[['ubar', 'divFy', 'psl']]
# hadmm.to_netcdf(f'{save_dir}/HadGEM3-GC31-MM_1.1_psl_ubar_divFy_daily-efp_r300.nc')

---

### IPSL-CM6A-LR

In [129]:
# mslp['IPSL-CM6A-LR']['time'] = daily_efp['IPSL-CM6A-LR']['time']
# mslp['IPSL-CM6A-LR'] = mslp['IPSL-CM6A-LR'].transpose('ens_ax', 'time', 'lat', 'lon')

# ipsl = xr.merge([daily_efp['IPSL-CM6A-LR'], mslp['IPSL-CM6A-LR']])
# ipsl = ipsl[['ubar', 'divFy', 'psl']]  

# ipsl.to_netcdf(f'{save_dir}/IPSL-CM6A-LR_1.1_psl_ubar_divFy_daily-efp_r200.nc')


---

### MIROC6

In [130]:
# mslp['MIROC6']['time'] = daily_efp['MIROC6']['time']
# mslp['MIROC6'] = mslp['MIROC6'].transpose('ens_ax', 'time', 'lat', 'lon', 'bnds')

# miroc = xr.merge([daily_efp['MIROC6'], mslp['MIROC6']])
# miroc = miroc[['ubar', 'divFy', 'psl']]  

# miroc.to_netcdf(f'{save_dir}/MIROC6_1.1_psl_ubar_divFy_daily-efp_r100.nc')

---

# Reanalysis

In [131]:
# jra55_mslp = xr.open_dataset('/home/links/ct715/data_storage/reanalysis/jra55_daily/variables/mslp/slp_monthly_jra55_1958_2016.nc')
# jra55_uvt = xr.open_dataset('/home/links/ct715/data_storage/reanalysis/jra55_daily/monthly/jra55_uvtw_ubar_ep_monthly.nc')

# jra55 = xr.merge([jra55_mslp, jra55_uvt])
# jra55 = jra55[['slp', 'ubar', 'div1']]
# jra55 = jra55.rename({'slp': 'psl', 'div1': 'divFy'})

# jra55.to_netcdf('/home/links/ct715/data_storage/reanalysis/jra55_daily/monthly/jra55_psl_ubar_divFy_monthly_1958-2016.nc')